# Case Study Model

## General Remarks

- Reverse Power Flow / Cost at substation
    - If the negative generation limit at the substation is not enforced, then substation has the option of pushing energy back towards the transmission system (reverse power flow)
    - We make the follwoing assumptions: 
        - reverse power flow is allowed at the substation 
        - for scheduled negative active power the substation is remunerated according to the market price (i.e. linear cost of energy provision at the substation) -> the substation has a linear cost function 
        - While the cost function will not have a quadratic termn, THERE WILL BE a penalty on $\alpha_0$ as it indicates how much the substation is willing to deviate from the schedule


## Notes and Todos:

- Verify the gamma decomposition
- Maybe check if 33bus gives nice/better results?
 
 
Done:   
- decide for a case! (combination of voltage limits and generators). Notes:
    - should there be an equal lambda case?
    - should the DERs be fully utilized in the deterministic case?
    - [Done]: Choosen Ex2 as set up
- put results handling in a function [done]
- remove unneccsray packages (e.g. gurobi from tomls) [done, but check on the "remains"]
- Remove quadratic term from substation [done]

In [1]:
# IJulia.qtconsole();

# set width of notbook 
display("text/html", "<style>.container { width:80% !important; }</style>")

In [143]:
# Set up environment from .toml-files
import Pkg
Pkg.activate(".")
# Pkg.instantiate()

# Load necessary packages
using DataFrames, CSV, LinearAlgebra, Dates # Basic Julia utility
using JLD, HDF5
using JuMP # Modeling Framework
using Mosek, MosekTools # Solver and Solver-Environment
using Logging # For some warnings in the functions

# Load functions and model
include("src/tools.jl") # Some additional functions
include("src/input.jl") # Type definitions and read-in functions
include("src/model_definition.jl") # Model definiton
include("src/output.jl") # Postprocessing of solved model

results_to_df (generic function with 1 method)

# 15Bus Case

- 15 bus case from Papavasilou paper
    - negative load is positive load
    - no line constraints
    - voltage at root is 1 p.u.
    
- Assumptions for all cases except mentioned differently
    - Two DERs one each at node 6 and 11 with `c1 = 10`, `c2 = 5`, `p_max = 1`, `q_max = 0.5`
    - No reverse power flow limit on substation
    - Substation pays `c1 = 50` for energy provision from the transmission network (no quadratic term) and a penalty of `200` on alpha to discourage high alpha

## Experiment 5

Same as Experiment 2 but with extended constraints (thermal for now)

In [61]:
experiment5 = Dict()

# Load feeder data 
feeder = load_feeder("data/feeder_data/basecase_noneg/")

# Set up
relative_std = 0.2
loads = [b.d_P for b in feeder.buses]
std_vec = abs.(loads) .* relative_std
var_vec = std_vec.^2
Σ = diagm(0 => var_vec)

# ex5_general_settings
ex5_general_settings = Dict(
    "var_vec" => var_vec,
    "Σ" => Σ,
    "z_g" => 1.945,
    "z_v" => 2.576,
    "toggle_volt_cc" => false,
    "toggle_gen_cc" => false,
    "toggle_thermal_cc" => false,
    "thermal_const_method" => 2, #1=soc, 2=inner approx
    "vfac" => 0.1, # directly sets symetric voltage limit to min:(1-vfac) max:(1+vfac), use import values if vfac=0
    "qcfac" => 1,
    "output_level" => 0,
    "Ψ" => 0,
)

# ex5_no_uncert:
ex5_no_uncert_settings = copy(ex5_general_settings)
ex5_no_uncert_settings["toggle_volt_cc"] = false
ex5_no_uncert_settings["toggle_gen_cc"] = false
ex5_no_uncert_settings["toggle_thermal_cc"] = false

experiment5["no_uncert"] = Dict(
    "verbose" => "no_uncert",
    "feeder" => feeder,
    "settings" => ex5_no_uncert_settings,
)

# ex5_uncert:
ex5_uncert_settings = copy(ex5_general_settings)
ex5_uncert_settings["toggle_volt_cc"] = true
ex5_uncert_settings["toggle_gen_cc"] = true
ex5_no_uncert_settings["toggle_thermal_cc"] = true
experiment5["uncert"] = Dict(
    "verbose" => "uncert",
    "feeder" => feeder,
    "settings" => ex5_uncert_settings,
)

# ex5_gen_uncert:
ex5_uncert_settings = copy(ex5_general_settings)
ex5_uncert_settings["toggle_volt_cc"] = false
ex5_uncert_settings["toggle_gen_cc"] = true
ex5_no_uncert_settings["toggle_thermal_cc"] = false
experiment5["gen_uncert"] = Dict(
    "verbose" => "gen_uncert",
    "feeder" => feeder,
    "settings" => ex5_uncert_settings,
)

# Run 
experiment5 = run_experiment(experiment5)

# Show results
sleep(.2) # just to ensure correct printing order
for case in keys(experiment5)
    print(">>> Case $case")
    sleep(.2)
    display(experiment5[case]["results"])
end

>>>>> Reading feeder data from data/feeder_data/basecase_noneg/
Running uncert
>>>> Building Model
>>>> Running Model
MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(86) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements are specified in sparse row ''(92) of matrix 'A'.
>>>> Model finished with status OPTIMAL in 0.020127179 seconds
>>>> Post-Processing

Running gen_uncert
>>>> Building Model
>>>> Running Model
>>>> Model finished with status OPTIMAL in 0.008015395 seconds
>>>> Post-Processing

Running no_uncert
>>>> Building Model
>>>> Running Model
>>>> Model finished with status OPTIMAL in 0.025528058 seconds
>>>> Post-Processing

>>> Case uncert

,objective,bus,gp,gq,voltage,fp,fq,alpha,lambda,pi,gamma,rho,eta,delta_plus,delta_minus,mu_plus,mu_minus,voltvar,lambda_anc,rx_pi_i,rx_pi_a,r_sum_mu_d,eta_calc
,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,87.2497,1,1.1833,0.3237,1.0,0,0,0.804262,-50.0,-6.91862e-8,-31.6204,0,0,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.976936,-0.0,-0.0,0,-50.0,-2.56748e-5,0.0,0.000901998,-2.73358e-8,0,0,-4.22673e-6,-1.28482e-5,0.0173874,-50.0,-2.13957e-7,-5.76552e-10,-2.27155e-7,-7.12928e-7
3,0.0,3,0,0,0.999807,0.5376,0.1855,0,-17.273,-8.7675,0.0,0.00953208,-2.43844e-6,0,0,-6.29109e-6,-8.54218e-6,0.0205741,-50.0,-6.13447,-1.79642e-5,-2.15803e-5,-5.89962e-5
4,0.0,4,0,0,1.03464,-0.256,1.08112e-7,0,-15.4536,-7.74441e-6,0.0,0.0230587,-6.14523e-6,0,0,-1.33399e-5,-4.58436e-6,0.0179232,-17.273,-5.41874e-6,-6.13459,-3.44478e-5,-0.000143992
5,0.0,5,0,0,1.03631,-0.256,1.08112e-7,0,-15.4536,-5.28285e-6,0.0,0.0230587,-6.26235e-6,0,0,-1.39036e-5,-4.41911e-6,0.0176355,-15.4536,-3.69606e-6,-5.41825e-6,-1.29318e-6,-0.000147459
6,0.0,6,0,0,1.03823,-0.0459445,-0.0311242,0,-15.4536,-4.12382e-6,0.0,0.0230587,-6.33796e-6,0,0,-1.45944e-5,-4.24663e-6,0.0172843,-15.4536,-2.87517e-6,-3.68326e-6,-8.52897e-7,-0.000149643
7,0.0,7,0.114245,0.0482242,1.04533,-0.0632445,-0.0354242,0.0977359,-15.4536,-3.69754e-7,0.0,0.0230587,-6.45472e-6,-1.31518e-6,-27.0348,-1.774e-5,-3.71932e-6,0.0158956,-15.4536,-2.58667e-7,-2.88488e-6,-1.35159e-6,-0.000152845
8,0.0,8,0,0,1.05208,-0.0923445,-0.0427242,0,-15.4536,-1.12339e-6,0.0,0.0321482,-7.29057e-6,0,0,-3.11552e-5,-3.647e-6,0.016132,-15.4536,-7.86525e-7,-2.09083e-6,-2.87736e-6,-0.000164977
9,0.0,9,0,0,1.04238,-0.1969,0.0019,0,-15.4536,-2.98633e-6,0.0,0.0270366,-6.91446e-6,0,0,-1.71109e-5,-4.14316e-6,0.0171313,-15.4536,-2.08838e-6,-5.41577e-6,-6.66191e-6,-0.000159341


>>> Case gen_uncert

,objective,bus,gp,gq,voltage,fp,fq,alpha,lambda,pi,gamma,rho,eta,delta_plus,delta_minus,mu_plus,mu_minus,voltvar,lambda_anc,rx_pi_i,rx_pi_a,r_sum_mu_d,eta_calc
,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,87.2497,1,1.1833,0.3237,1.0,0,0,0.804262,-50.0,-2.09325e-8,-31.6204,0,0,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.976936,0.5376,0.1855,0,-50.0,-2.04872e-6,0.0,0,0,0,0,-1.21123e-6,-2.29922e-6,0,-50.0,-1.70727e-8,-1.74438e-10,-1.72471e-8,0
3,0.0,3,0,0,0.999807,-0.256,-4.85054e-9,0,-17.2732,-8.76751,0.0,0,0,0,0,-1.51743e-6,-1.81489e-6,0,-50.0,-6.13448,-1.43345e-6,-1.71506e-6,0
4,0.0,4,0,0,1.03464,-0.256,-4.85054e-9,0,-15.4536,-8.7742e-7,0.0,0,0,0,0,-2.25493e-6,-1.38053e-6,0,-17.2732,-6.13928e-7,-6.1346,-2.77049e-6,0
5,0.0,5,0,0,1.03568,-0.0459906,-0.00737375,0,-15.4536,-6.54951e-7,0.0,0,0,0,0,-2.2954e-6,-1.36379e-6,0,-15.4536,-4.58226e-7,-6.13873e-7,-1.23797e-7,0
6,0.0,6,0,0,1.03703,-0.0632906,-0.0116738,0,-15.4536,-5.39032e-7,0.0,0,0,0,0,-2.34755e-6,-1.34435e-6,0,-15.4536,-3.75819e-7,-4.56639e-7,-8.08202e-8,0
7,0.0,7,0.114291,0.0244738,1.04257,-0.0923906,-0.0189738,0.0977752,-15.4536,-8.44846e-8,0.0,0,0,-5.66531e-7,-27.0348,-2.58078e-6,-1.27483e-6,0,-15.4536,-5.91024e-8,-3.77088e-7,-1.25893e-7,0
8,0.0,8,0,0,1.05339,-0.1969,0.0019,0,-15.4536,-5.84504e-8,0.0,0,0,0,0,-3.06224e-6,0.0,0,-15.4536,-4.09231e-8,-3.61234e-7,-3.20311e-7,0
9,0.0,9,0,0,1.04371,-0.230109,-0.00102625,0,-15.4536,-5.1595e-7,0.0,0,0,0,0,0.0,-1.29619e-6,0,-15.4536,-3.6081e-7,-6.13591e-7,-4.79759e-7,0


>>> Case no_uncert

,objective,bus,gp,gq,voltage,fp,fq,alpha,lambda,pi,gamma,rho,eta,delta_plus,delta_minus,mu_plus,mu_minus,voltvar,lambda_anc,rx_pi_i,rx_pi_a,r_sum_mu_d,eta_calc
,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,61.6704,1,1.1833,0.3237,1.0,0,0,0,-50.0,-1.48341e-7,0.0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.976936,0.5376,0.1855,0,-50.0,-1.88026e-5,0.0,0,0,0,0,-1.02162e-5,-1.89252e-5,0,-50.0,-1.56688e-7,-1.23617e-9,-1.57924e-7,0
3,0.0,3,0,0,0.999807,-0.256,2.1557e-8,0,-30.475,-5.23072,0.0,0,0,0,0,-1.28548e-5,-1.44282e-5,0,-50.0,-3.65985,-1.31558e-5,-1.54827e-5,0
4,0.0,4,0,0,1.03464,-0.256,2.1557e-8,0,-10.9503,-4.06447e-6,0.0,0,0,0,0,-1.87577e-5,-1.06553e-5,0,-30.475,-2.84389e-6,-3.65992,-2.47029e-5,0
5,0.0,5,0,0,1.03572,-0.0457604,-0.0091495,0,-10.9503,-2.11379e-7,0.0,0,0,0,0,-1.91103e-5,-1.05291e-5,0,-10.9503,-1.47888e-7,-2.84364e-6,-1.12066e-6,0
6,0.0,6,0,0,1.03711,-0.0630604,-0.0134495,0,-10.9503,-8.30544e-7,0.0,0,0,0,0,-1.95532e-5,-1.03829e-5,0,-10.9503,-5.79064e-7,-1.47376e-7,-7.2644e-7,0
7,0.0,7,0.11406,0.0262495,1.04276,-0.0921604,-0.0207495,0,-10.9502,-6.39343e-6,0.0,0,0,0.0,-2.29822e-5,-2.14456e-5,-9.86039e-6,0,-10.9503,-4.47261e-6,-5.81019e-7,-1.11681e-6,0
8,0.0,8,0,0,1.0533,-0.1969,0.0019,0,-10.9501,-1.3063e-5,0.0,0,0,0,0,-2.51783e-5,0.0,0,-10.9501,-9.14584e-6,-6.51219e-6,-2.63365e-6,0
9,0.0,9,0,0,1.04362,-0.23034,0.000749525,0,-10.9501,-9.30136e-6,0.0,0,0,0,0,0.0,-9.99776e-6,0,-10.9503,-6.50456e-6,-2.84233e-6,-4.21697e-6,0


## Experiment 1

We analyse the influence of the uncertainty in the system using the "standard" setup

### Description

Ex1:
- tag: `experiment1`
- tight bascase voltage (0.95-1.05) 
- all nodes have 20% standard deviation of their forecasted load

Runs:
- `ex1_no_uncert`: deterministic case
- `ex1_uncert`: case with uncertainty
- `ex1_gen_uncert`: Chance constraints only in generation


### Notes and Observations

- Recursive decomposition of prices for active power (eqtns (20) and (21)) are verified by the experimet
- Decompositon via voltage constraints (eqtns (23) and (24)) also verified by the experiment

In [36]:
experiment1 = Dict()

# Load feeder data 
feeder = load_feeder("data/feeder_data/basecase_lv_noneg/")

# Set up
relative_std = 0.2
loads = [b.d_P for b in feeder.buses]
std_vec = loads .* relative_std
var_vec = std_vec.^2
Σ = diagm(0 => var_vec)

# ex1_general_settings
ex1_general_settings = Dict(
    "var_vec" => var_vec,
    "Σ" => Σ,
    "z_g" => 1.945,
    "z_v" => 2.576,
    "toggle_volt_cc" => false,
    "toggle_gen_cc" => false,
    "vfac" => 0, # directly sets symetric voltage limit to min:(1-vfac) max:(1+vfac), use import values if vfac=0
    "qcfac" => 1,
    "output_level" => 0,
    "Ψ" => 0,
)


# ex1_no_uncert:
ex1_no_uncert_settings = copy(ex1_general_settings)
ex1_no_uncert_settings["toggle_volt_cc"] = false
ex1_no_uncert_settings["toggle_gen_cc"] = false

experiment1["no_uncert"] = Dict(
    "verbose" => "ex1_no_uncert",
    "feeder" => feeder,
    "settings" => ex1_no_uncert_settings,
)

# ex1_uncert:
ex1_uncert_settings = copy(ex1_general_settings)
ex1_uncert_settings["toggle_volt_cc"] = true
ex1_uncert_settings["toggle_gen_cc"] = true
experiment1["uncert"] = Dict(
    "verbose" => "ex1_uncert",
    "feeder" => feeder,
    "settings" => ex1_uncert_settings,
)

# ex1_gen_uncert:
ex1_uncert_settings = copy(ex1_general_settings)
ex1_uncert_settings["toggle_volt_cc"] = false
ex1_uncert_settings["toggle_gen_cc"] = true
experiment1["gen_uncert"] = Dict(
    "verbose" => "ex1_gen_uncert",
    "feeder" => feeder,
    "settings" => ex1_uncert_settings,
)

# Run 
experiment1 = run_experiment(experiment1)

# Show results
sleep(.2) # just to ensure correct printing order
for case in keys(experiment1)
    display(experiment1[case]["results"])
end

>>>>> Reading feeder data from data/feeder_data/basecase_lv_noneg/
Running ex1_uncert
>>>> Building Model


KeyError: KeyError: key "toggle_thermal" not found

In [3]:
save_experiment("experiment1", experiment1)

Saving ex1_uncert
Saving ex1_gen_uncert
Saving ex1_no_uncert
>>>> Saved with timestamp 190320_0927


## Experiment 2

Same as Experiment1 but with wider voltage constraints

### Description

Ex2:
- tag: `experiment2` 
- bascase voltage (0.9-1.1) and `vfac = 0.02`: Resulting "real" voltage limits: ca. 0.91 - 1.09 (i.e. +- 9%)
- all nodes have 20% standard deviation of their forecasted load
- all parameters as in AP's paper, including the negative load (i.e. uncontrollable generator with the same standard deviation
- substation balancing participation penalty is set to 1000 (i.e. quadratic cost term)

Runs:
- `ex2_no_uncert`: deterministic case
- `ex2_uncert`: case with uncertainty
- `ex2_gen_uncert`: chance constraints only in generation


### Notes and Observations

- Without uncertainty the network is "uncongested" so that price for active power is the same at all nodes
- With uncertainty, some voltage constraints become binding and there are differences in the prices

In [49]:
experiment2 = Dict()

# Load feeder data 
feeder = load_feeder("data/feeder_data/basecase_noneg/")

# Set up
relative_std = 0.2
loads = [b.d_P for b in feeder.buses]
std_vec = abs.(loads) .* relative_std
var_vec = std_vec.^2
Σ = diagm(0 => var_vec)

# ex2_general_settings
ex2_general_settings = Dict(
    "var_vec" => var_vec,
    "Σ" => Σ,
    "z_g" => 1.945,
    "z_v" => 2.576,
    "toggle_volt_cc" => false,
    "toggle_gen_cc" => false,
    "thermal_const_method" => 0,
    "vfac" => 0.1, # directly sets symetric voltage limit to min:(1-vfac) max:(1+vfac), use import values if vfac=0
    "qcfac" => 1,
    "output_level" => 0,
    "Ψ" => 0,
)

# ex2_no_uncert:
ex2_no_uncert_settings = copy(ex2_general_settings)
ex2_no_uncert_settings["toggle_volt_cc"] = false
ex2_no_uncert_settings["toggle_gen_cc"] = false

experiment2["no_uncert"] = Dict(
    "verbose" => "no_uncert",
    "feeder" => feeder,
    "settings" => ex2_no_uncert_settings,
)

# ex2_uncert:
ex2_uncert_settings = copy(ex2_general_settings)
ex2_uncert_settings["toggle_volt_cc"] = true
ex2_uncert_settings["toggle_gen_cc"] = true
experiment2["uncert"] = Dict(
    "verbose" => "uncert",
    "feeder" => feeder,
    "settings" => ex2_uncert_settings,
)

# ex2_gen_uncert:
ex2_uncert_settings = copy(ex2_general_settings)
ex2_uncert_settings["toggle_volt_cc"] = false
ex2_uncert_settings["toggle_gen_cc"] = true
experiment2["gen_uncert"] = Dict(
    "verbose" => "gen_uncert",
    "feeder" => feeder,
    "settings" => ex2_uncert_settings,
)

# Run 
experiment2 = run_experiment(experiment2)

# Show results
sleep(.2) # just to ensure correct printing order
for case in keys(experiment2)
    print(">>> Case $case")
    sleep(.2)
    display(experiment2[case]["results"])
end

>>>>> Reading feeder data from data/feeder_data/basecase_noneg/
Running uncert
>>>> Building Model
>>>> Running Model
MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(86) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements are specified in sparse row ''(92) of matrix 'A'.
>>>> Model finished with status OPTIMAL in 0.031447259 seconds
>>>> Post-Processing

Running gen_uncert
>>>> Building Model
>>>> Running Model
>>>> Model finished with status OPTIMAL in 0.004507253 seconds
>>>> Post-Processing

Running no_uncert
>>>> Building Model
>>>> Running Model
>>>> Model finished with status OPTIMAL in 0.003282563 seconds
>>>> Post-Processing

>>> Case uncert

,objective,bus,gp,gq,voltage,fp,fq,alpha,lambda,pi,gamma,rho,eta,delta_plus,delta_minus,mu_plus,mu_minus,voltvar,lambda_anc,rx_pi_i,rx_pi_a,r_sum_mu_d,eta_calc
,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,42.1519,1,-0.0653161,0.933285,1.0,0,0,0.89513,-50.0,0.0,-31.6737,0,0,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.900334,-0.0,-0.0,0,-50.137,-16.4398,0.0,0.000934452,-0.0515086,0,0,-2.93471e-7,-91.1231,0.000116789,-50.0,-0.136999,0.0,0.136999,-0.35219
3,0.0,3,0,0,0.960447,-0.711016,0.795085,0,-46.1416,-10.7296,0.0,0.00440658,-0.0597499,0,0,-5.47177e-7,-3.1151e-6,0.0142634,-50.137,-7.5073,-11.5027,-3.99537,-6.53755
4,0.0,4,0,0,1.04775,-1.50462,0.609585,0,-39.8793,-1.77957,0.0,0.00984874,-0.0726673,0,0,-3.42637e-6,-5.94266e-7,0.0161622,-46.1416,-1.24516,-7.50744,-6.26228,-16.2324
5,0.0,5,0,0,1.05393,-1.50462,0.609585,0,-39.5991,-1.37906,0.0,0.00984874,-0.0727774,0,0,-4.31794e-6,-5.40652e-7,0.014704,-39.8793,-0.964836,-1.24505,-0.280213,-16.6662
6,0.0,6,0,0,1.05993,-0.685736,0.241882,0,-39.3424,-1.01082,0.0,0.00984874,-0.0728783,0,0,-5.753e-6,-4.96476e-7,0.0132576,-39.5991,-0.704755,-0.961495,-0.25674,-17.0637
7,0.0,7,0.754036,-0.224782,1.0781,-0.703036,0.237582,0.0393219,-38.6353,-8.3424e-7,0.0,0.00984874,-0.0731561,-27.0217,-1.21731e-7,-7.33542,-3.96447e-7,0.00925915,-39.3424,-5.83605e-7,-0.707134,-0.707135,-18.1584
8,0.0,8,0,0,1.06984,-0.732136,0.230282,0,-37.0357,-2.28408,0.0,0.0135057,-0.0808108,0,0,-15.2884,-4.68523e-7,0.0127008,-38.6349,-1.59916,-1.45671e-6,-1.59916,-20.6347
9,0.0,9,0,0,1.06031,-0.1969,0.0019,0,-38.6349,-2.08062e-6,0.0,0.0114491,-0.0762313,0,0,-7.04568e-6,-5.1421e-7,0.0135812,-39.8793,-1.455e-6,-1.24448,-1.24448,-18.159


>>> Case gen_uncert

,objective,bus,gp,gq,voltage,fp,fq,alpha,lambda,pi,gamma,rho,eta,delta_plus,delta_minus,mu_plus,mu_minus,voltvar,lambda_anc,rx_pi_i,rx_pi_a,r_sum_mu_d,eta_calc
,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,40.8639,1,-0.178548,0.936735,1.0,0,0,0.991999,-50.0,0.0,-31.6228,0,0,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.9,-0.824248,0.798535,0,-50.137,-16.4434,0.0,0,0,0,0,-1.72035e-7,-89.8818,0,-50.0,-0.137028,0.0,0.137028,0
3,0.0,3,0,0,0.970042,-1.61785,0.613035,0,-46.3635,-11.0502,0.0,0,0,0,0,-2.54553e-7,-5.48936e-7,0,-50.137,-7.73166,-11.5052,-3.77352,0
4,0.0,4,0,0,1.07065,-1.61785,0.613035,0,-40.449,-2.59722,0.0,0,0,0,0,-9.59398e-7,-2.23593e-7,0,-46.3635,-1.81726,-7.73181,-5.91455,0
5,0.0,5,0,0,1.07661,-0.726788,0.273872,0,-40.04,-2.01268,0.0,0,0,0,0,-1.17303e-6,-2.15951e-7,0,-40.449,-1.40814,-1.8171,-0.40896,0
6,0.0,6,0,0,1.08241,-0.744088,0.269572,0,-39.6653,-1.47525,0.0,0,0,0,0,-1.5519e-6,-2.08817e-7,0,-40.04,-1.02856,-1.40326,-0.374702,0
7,0.0,7,0.795088,-0.256772,1.1,-0.773188,0.262272,0.00420203,-38.6333,-2.55501e-7,0.0,0,0,-27.0526,-8.45117e-8,-10.7058,-1.88918e-7,0,-39.6653,-1.7874e-7,-1.03204,-1.03204,0
8,0.0,8,0,0,1.09648,-0.1969,0.0019,0,-39.5811,-1.35618,0.0,0,0,0,0,-2.43789e-5,-0.0,0,-39.5811,-0.949505,-0.949508,-2.55003e-6,0
9,0.0,9,0,0,1.08718,-0.91116,0.330763,0,-39.5811,-1.35618,0.0,0,0,0,0,-0.0,-1.99028e-7,0,-40.449,-0.948394,-1.81627,-0.867873,0


>>> Case no_uncert

,objective,bus,gp,gq,voltage,fp,fq,alpha,lambda,pi,gamma,rho,eta,delta_plus,delta_minus,mu_plus,mu_minus,voltvar,lambda_anc,rx_pi_i,rx_pi_a,r_sum_mu_d,eta_calc
,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,8.49057,1,-0.178548,0.936735,1.0,0,0,0,-50.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.9,-0.824248,0.798535,0,-50.4708,-56.4929,0.0,0,0,0,0,-0.0,-308.797,0,-50.0,-0.470774,0.0,0.470774,0
3,0.0,3,0,0,0.970042,-1.61785,0.613035,0,-37.5065,-37.9641,0.0,0,0,0,0,-5.98486e-7,-1.23903e-6,0,-50.4708,-26.5628,-39.5271,-12.9643,0
4,0.0,4,0,0,1.07065,-1.61785,0.613035,0,-17.1865,-8.92298,0.0,0,0,0,0,-2.74113e-6,-4.58805e-7,0,-37.5065,-6.24338,-26.5634,-20.32,0
5,0.0,5,0,0,1.07661,-0.725632,0.273064,0,-15.7815,-6.91476,0.0,0,0,0,0,-3.47323e-6,-4.40339e-7,0,-17.1865,-4.8378,-6.24282,-1.40502,0
6,0.0,6,0,0,1.08241,-0.742932,0.268764,0,-14.4942,-5.06837,0.0,0,0,0,0,-4.64203e-6,-4.23712e-7,0,-15.7815,-3.53373,-4.82105,-1.28732,0
7,0.0,7,0.793932,-0.255964,1.1,-0.772032,0.261464,0,-10.9485,-9.33885e-7,0.0,0,0,-4.86351e-5,-1.82238e-7,-36.7806,-3.798e-7,0,-14.4942,-6.53313e-7,-3.54565,-3.54565,0
8,0.0,8,0,0,1.09648,-0.1969,0.0019,0,-14.2049,-4.65928,0.0,0,0,0,0,-3.01682e-5,-0.0,0,-14.2049,-3.26212,-3.26213,-3.15559e-6,0
9,0.0,9,0,0,1.08718,-0.912316,0.331571,0,-14.2049,-4.65929,0.0,0,0,0,0,-0.0,-4.15153e-7,0,-17.1865,-3.2583,-6.23996,-2.98166,0


In [44]:
save_experiment("experiment2", experiment2)

Saving uncert
Saving gen_uncert
Saving no_uncert
>>>> Saved with timestamp 190320_1040


## Experiment 3

Same as before but instead of tighter voltage limits try higher load

In [10]:
experiment3 = Dict()

# Load feeder data 
feeder = load_feeder("data/feeder_data/basecase_noneg/")

# Set up
relative_std = 0.2
loads = [b.d_P for b in feeder.buses]
std_vec = loads .* relative_std
var_vec = std_vec.^2
Σ = diagm(0 => var_vec)

# ex3_general_settings
ex3_general_settings = Dict(
    "var_vec" => var_vec,
    "Σ" => Σ,
    "z_g" => 1.945,
    "z_v" => 2.576,
    "toggle_volt_cc" => false,
    "toggle_gen_cc" => false,
    "vfac" => 0, # decrease upper bound by factor (1-vfac) and increase lower bound by (1+vfac)
    "qcfac" => 1,
    "output_level" => 0,
    "Ψ" => 0,
    "loadfac" => 3
)

# ex3_no_uncert:
ex3_no_uncert_settings = copy(ex3_general_settings)
ex3_no_uncert_settings["toggle_volt_cc"] = false
ex3_no_uncert_settings["toggle_gen_cc"] = false

experiment3["no_uncert"] = Dict(
    "verbose" => "ex3_no_uncert",
    "feeder" => feeder,
    "settings" => ex3_no_uncert_settings,
)

# ex3_uncert:
ex3_uncert_settings = copy(ex3_general_settings)
ex3_uncert_settings["toggle_volt_cc"] = true
ex3_uncert_settings["toggle_gen_cc"] = true
experiment3["uncert"] = Dict(
    "verbose" => "ex3_uncert",
    "feeder" => feeder,
    "settings" => ex3_uncert_settings,
)

# ex3_gen_uncert:
ex3_uncert_settings = copy(ex3_general_settings)
ex3_uncert_settings["toggle_volt_cc"] = false
ex3_uncert_settings["toggle_gen_cc"] = true
experiment3["gen_uncert"] = Dict(
    "verbose" => "ex3_gen_uncert",
    "feeder" => feeder,
    "settings" => ex3_uncert_settings,
)

# Run 
experiment3 = run_experiment(experiment3)

# Show results
sleep(.2) # just to ensure correct printing order
for case in keys(experiment3)
    print(">>> Case $case")
    sleep(.2)
    display(experiment3[case]["results"])
end

>>>>> Reading feeder data from data/feeder_data/basecase_noneg/
Running ex3_uncert
>>>> Building Model
>>>> Running Model
MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(87) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements are specified in sparse row ''(93) of matrix 'A'.
>>>> Model finished with status OPTIMAL in 0.006354567 seconds
>>>> Post-Processing

Running ex3_gen_uncert
>>>> Building Model
>>>> Running Model
>>>> Model finished with status OPTIMAL in 0.002476817 seconds
>>>> Post-Processing

Running ex3_no_uncert
>>>> Building Model
>>>> Running Model
>>>> Model finished with status OPTIMAL in 0.00223549 seconds
>>>> Post-Processing

>>> Case uncert

,objective,bus,gp,gq,voltage,alpha,lambda,pi,gamma,eta,delta_plus,delta_minus,mu_plus,mu_minus,voltvar,lambda_anc,rx_pi_i,rx_pi_a,r_sum_mu_d
,Float64,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,431.923,1,3.8177,1.0676,1.0,1.0,-100.0,-7.68782e-10,-31.6228,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.916252,0,-100.0,-0.000145897,0.0,-1.5754e-8,0,0,-3.26811e-6,-0.000764702,0.00363129,-100.0,-1.21581e-6,-6.40651e-12,1.1877e-6
3,0.0,3,0,0,0.950524,0,-100.0,-0.000106729,0.0,-5.32418e-6,0,0,-4.5928e-6,-5.34742e-5,0.0117104,-100.0,-7.46768e-5,-0.000102082,-2.95957e-5
4,0.0,4,0,0,1.00189,0,-99.9999,-2.54546e-5,0.0,-1.30342e-5,0,0,-1.39688e-5,-1.24571e-5,0.0211438,-100.0,-1.78105e-5,-7.46783e-5,-5.99181e-5
5,0.0,5,0,0,1.0105,0,-99.9999,-1.88947e-5,0.0,-1.3443e-5,0,0,-1.80855e-5,-9.65875e-6,0.0212127,-99.9999,-1.32194e-5,-1.78089e-5,-4.72767e-6
6,0.0,6,0,0,1.01952,0,-99.9999,-1.32239e-5,0.0,-1.37819e-5,0,0,-2.39752e-5,-7.56658e-6,0.0205562,-99.9999,-9.21983e-6,-1.31736e-5,-4.0367e-6
7,0.0,7,0.8,-0.0477213,1.04946,0,-99.9999,-2.49905e-7,0.0,-1.45891e-5,-88.4176,-2.06216e-6,-0.000103338,-4.4126e-6,0.0150349,-99.9999,-1.74825e-7,-9.25095e-6,-9.53643e-6
8,0.0,8,0,0,0.970333,0,-99.9999,-2.22249e-5,0.0,-1.47582e-5,0,0,-7.50706e-6,-5.08665e-5,0.0177988,-99.9999,-1.55604e-5,-1.08896e-5,4.53539e-6
9,0.0,9,0,0,1.00209,0,-99.9999,-1.55536e-5,0.0,-1.43558e-5,0,0,-1.67668e-5,-1.18817e-5,0.0209381,-99.9999,-1.08768e-5,-1.78007e-5,-7.42324e-6


>>> Case gen_uncert

,objective,bus,gp,gq,voltage,alpha,lambda,pi,gamma,eta,delta_plus,delta_minus,mu_plus,mu_minus,voltvar,lambda_anc,rx_pi_i,rx_pi_a,r_sum_mu_d
,Float64,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,431.923,1,3.8177,1.06751,1.0,1.0,-100.0,0.0,-31.6228,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.916265,0,-100.0,-1.46431e-5,0.0,0,0,0,-5.19691e-7,-7.75417e-5,0,-100.0,-1.22026e-7,0.0,1.22026e-7
3,0.0,3,0,0,0.950548,0,-100.0,-1.06024e-5,0.0,0,0,0,-6.95592e-7,-3.72078e-6,0,-100.0,-7.41829e-6,-1.02455e-5,-2.82723e-6
4,0.0,4,0,0,1.00193,0,-100.0,-3.07235e-6,0.0,0,0,0,-1.49108e-6,-9.49878e-7,0,-100.0,-2.14971e-6,-7.41843e-6,-5.26872e-6
5,0.0,5,0,0,1.01167,0,-100.0,-2.38737e-6,0.0,0,0,0,-1.87868e-6,-8.13124e-7,0,-100.0,-1.67029e-6,-2.14952e-6,-4.79231e-7
6,0.0,6,0,0,1.02172,0,-100.0,-1.81109e-6,0.0,0,0,0,-2.49437e-6,-7.01822e-7,0,-100.0,-1.26271e-6,-1.6645e-6,-4.01791e-7
7,0.0,7,0.8,-0.00573882,1.05434,0,-100.0,-4.76193e-7,0.0,0,-88.4189,-3.44537e-7,-1.01621e-5,-4.74925e-7,0,-100.0,-3.33128e-7,-1.26698e-6,-9.33847e-7
8,0.0,8,0,0,0.96786,0,-100.0,-2.67144e-6,0.0,0,0,0,-0.0,-1.95064e-6,0,-100.0,-1.87037e-6,-1.66633e-6,2.04037e-7
9,0.0,9,0,0,0.999696,0,-100.0,-2.38002e-6,0.0,0,0,0,-1.41964e-6,-0.0,0,-100.0,-1.66437e-6,-2.14853e-6,-4.84157e-7


>>> Case no_uncert

,objective,bus,gp,gq,voltage,alpha,lambda,pi,gamma,eta,delta_plus,delta_minus,mu_plus,mu_minus,voltvar,lambda_anc,rx_pi_i,rx_pi_a,r_sum_mu_d
,Float64,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,399.29,1,3.8177,1.07126,1.0,0,-100.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.915774,0,-100.0,-7.20018e-6,0.0,0,0,0,-0.0,-3.84094e-5,0,-100.0,-6.00015e-8,0.0,6.00015e-8
3,0.0,3,0,0,0.949578,0,-100.0,-5.07783e-6,0.0,0,0,0,-2.51789e-7,-1.51122e-6,0,-100.0,-3.55287e-6,-5.03784e-6,-1.48497e-6
4,0.0,4,0,0,1.00027,0,-100.0,-1.25314e-6,0.0,0,0,0,-6.13942e-7,-0.0,0,-100.0,-8.76815e-7,-3.55294e-6,-2.67613e-6
5,0.0,5,0,0,1.01009,0,-100.0,-9.36669e-7,0.0,0,0,0,-7.81308e-7,-3.25326e-7,0,-100.0,-6.55326e-7,-8.76736e-7,-2.21411e-7
6,0.0,6,0,0,1.02022,0,-100.0,-6.68596e-7,0.0,0,0,0,-1.05106e-6,-2.82462e-7,0,-100.0,-4.66152e-7,-6.53056e-7,-1.86904e-7
7,0.0,7,0.8,-0.00323173,1.05305,0,-100.0,-3.86404e-8,0.0,0,-89.0495,-8.43607e-8,-4.76745e-6,-1.9593e-7,0,-100.0,-2.70315e-8,-4.67726e-7,-4.40694e-7
8,0.0,8,0,0,0.965763,0,-100.0,-9.90414e-7,0.0,0,0,0,-0.0,-7.79882e-7,0,-100.0,-6.93422e-7,-6.11847e-7,8.15756e-8
9,0.0,9,0,0,0.997666,0,-100.0,-8.739e-7,0.0,0,0,0,-6.07748e-7,-0.0,0,-100.0,-6.11129e-7,-8.76334e-7,-2.65205e-7


## Experiment 4

- Instead of uncertain load uncontrollable vs. controllable DERs
- Maybe better in the 33-bus system?

## Prototyping Space

In [5]:
res = experiment2["uncert"]["results"]

,objective,bus,gp,gq,voltage,alpha,lambda,pi,gamma,eta,delta_plus,delta_minus,mu_plus,mu_minus,voltvar,lambda_anc,rx_pi_i,rx_pi_a,r_sum_mu_d,eta_calc
,Float64,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,42.1519,1,-0.0653151,0.933284,1.0,0.895129,-50.0,-3.34394e-9,-31.6735,0,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.900334,0,-50.137,-16.4401,0.0,-0.0512537,0,0,-7.25317e-8,-91.1244,0.000116804,-50.0,-0.137001,-2.78662e-11,0.137001,-0.352195
3,0.0,3,0,0,0.960447,0,-46.1416,-10.7297,0.0,-0.0596395,0,0,-1.34272e-7,-7.07984e-7,0.0139108,-50.137,-7.5074,-11.5028,-3.99543,-6.53764
4,0.0,4,0,0,1.04775,0,-39.8792,-1.7796,0.0,-0.0727832,0,0,-8.1725e-7,-1.40745e-7,0.0151213,-46.1416,-1.24518,-7.50755,-6.26237,-16.2326
5,0.0,5,0,0,1.05393,0,-39.599,-1.37908,0.0,-0.0729688,0,0,-1.00029e-6,-1.2926e-7,0.0138768,-39.8792,-0.96485,-1.24507,-0.280217,-16.6664
6,0.0,6,0,0,1.05993,0,-39.3422,-1.01084,0.0,-0.0731388,0,0,-1.28435e-6,-1.19442e-7,0.0126278,-39.599,-0.704766,-0.961509,-0.256743,-17.0639
7,0.0,7,0.753981,-0.224743,1.0781,0.0393694,-38.6351,-1.166e-7,0.0,-0.0736071,-27.0216,-4.12736e-8,-7.33553,-9.70597e-8,0.00925916,-39.3422,-8.15695e-8,-0.707145,-0.707145,-18.1587
8,0.0,8,0,0,1.06984,0,-37.0355,-2.28412,0.0,-0.0807118,0,0,-15.2886,-1.08976e-7,0.0127008,-38.6347,-1.59919,-1.66564e-7,-1.59919,-20.635
9,0.0,9,0,0,1.06031,0,-38.6347,-2.37903e-7,0.0,-0.0762531,0,0,-1.49742e-6,-1.19298e-7,0.0131609,-39.8792,-1.66369e-7,-1.2445,-1.2445,-18.1593


In [62]:
feeder = experiment5["uncert"]["feeder"]
results = experiment5["uncert"]["results"]
sq_sig = (experiment5["uncert"]["settings"]["Σ"])[2:end,2:end]^(1/2)
e = ones(14)
rho = results[:rho][2:end]


Rd = feeder.R
A = feeder.A[1:end, 2:end]
R = A'*Rd*A
sq_sig[2,2] = 1e-6

Y = (R + rho*e')sq_sig




14×14 Array{Float64,2}:
 0.000301885  1.902e-9     7.64603e-6   …  2.52559e-7  4.04095e-6 
 0.00167165   9.88321e-8   0.000397305     2.66898e-6  4.27037e-5 
 0.0038186    1.12359e-7   0.00100805      6.45645e-6  0.000103303
 0.0038186    1.12359e-7   0.00100805      6.45645e-6  0.000103303
 0.0038186    1.12359e-7   0.00100805      6.45645e-6  0.000103303
 0.0038186    1.12359e-7   0.00100805   …  6.45645e-6  0.000103303
 0.00526128   1.21448e-7   0.00104459      9.00149e-6  0.000144024
 0.00444997   1.16337e-7   0.00102404      7.57025e-6  0.000121124
 0.00444997   1.16337e-7   0.00102404      7.57025e-6  0.000121124
 0.00444997   1.16337e-7   0.00102404      7.57025e-6  0.000121124
 0.00444997   1.16337e-7   0.00102404   …  7.57025e-6  0.000121124
 1.55549e-5   9.80019e-11  3.93968e-7      3.07441e-7  4.91905e-6 
 1.55549e-5   9.80019e-11  3.93968e-7      4.39594e-5  0.000703351
 1.55549e-5   9.80019e-11  3.93968e-7      4.39594e-5  0.0011303  

In [137]:
get_lPi_dk(feeder, results, 4, 5)

-0.06559285153403285

In [144]:
feeder = experiment5["uncert"]["feeder"]
results = experiment5["uncert"]["results"]
root_bus = feeder.root_bus
n_buses = feeder.n_buses

non_root_buses = setdiff(collect(1:n_buses), [root_bus])

LF_P = zeros(n_buses,n_buses)
LF_Q = zeros(n_buses,n_buses)
for i in non_root_buses
    for k in non_root_buses
        LF_P[i,k] = get_lPi_dk(feeder, results, i, k)
        LF_Q[i,k] = get_lQi_dk(feeder, results, i, k)
    end
end

DataFrame(LF_P)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
3,0.0,0.0,0.127749,0.118361,0.117155,0.116827,0.127749,0.127749,0.116426,0.115928,0.115688,0.127749,0.0,0.0,0.0
4,0.0,0.0,0.0,-0.0661956,-0.0655929,-0.0654287,-0.0661956,-0.0661956,-0.0652286,-0.0649797,-0.0648596,-0.0661956,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,-0.00910599,-0.00908321,-0.00910599,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,-0.0,-0.0,-0.0,-0.0,-0.00250241,-0.00250241,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
7,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.00870465,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
8,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.012764,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0146084,-0.0146084,-0.0145529,-0.0145261,-0.0146084,0.0,0.0,0.0


In [46]:
A = feeder.A[:,2:end]
A_check = A^-1

14×14 Array{Float64,2}:
 1.0  -1.0   0.0  -0.0   0.0  -0.0  …   0.0  -0.0   0.0  -0.0   0.0  -0.0
 0.0   1.0  -1.0  -0.0   0.0  -0.0      0.0  -0.0   0.0  -0.0   0.0  -0.0
 0.0   0.0   1.0  -1.0   0.0  -0.0      0.0  -0.0   0.0  -0.0   0.0  -0.0
 0.0   0.0   0.0   1.0  -1.0  -0.0      0.0  -0.0   0.0  -0.0   0.0  -0.0
 0.0   0.0   0.0   0.0   1.0  -1.0      0.0  -0.0   0.0  -0.0   0.0  -0.0
 0.0   0.0   0.0   0.0   0.0   1.0  …   0.0  -0.0   0.0  -0.0   0.0  -0.0
 0.0   0.0   0.0   0.0   0.0   0.0      0.0  -0.0   0.0  -0.0   0.0  -0.0
 0.0   0.0   0.0   0.0   0.0   0.0     -1.0  -0.0   0.0  -0.0   0.0  -0.0
 0.0   0.0   0.0   0.0   0.0   0.0      1.0  -1.0   0.0  -0.0   0.0  -0.0
 0.0   0.0   0.0   0.0   0.0   0.0      0.0   1.0  -1.0  -0.0   0.0  -0.0
 0.0   0.0   0.0   0.0   0.0   0.0  …   0.0   0.0   1.0  -0.0   0.0  -0.0
 0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   1.0  -1.0  -0.0
 0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   1.0  -1.0
 0.0   0.0   0

In [120]:
function traverse(feeder, bus_idx; nodes=[])
    push!(nodes, bus_idx)
    if length(feeder.buses[bus_idx].children) > 0
        for c in feeder.buses[bus_idx].children
            nodes = traverse(feeder, c, nodes=nodes)
        end
    end
    return nodes 
end

traverse (generic function with 2 methods)

In [47]:
Rd = feeder.R
A = feeder.A[1:end, 2:end]
R = A'*Rd*A




14×14 Array{Float64,2}:
 0.001  0.001   0.001   0.001   0.001   …  0.001   0.0    0.0     0.0   
 0.001  0.0893  0.0893  0.0893  0.0893     0.0893  0.0    0.0     0.0   
 0.001  0.0893  0.2277  0.2277  0.2277     0.2277  0.0    0.0     0.0   
 0.001  0.0893  0.2277  0.2468  0.2468     0.2277  0.0    0.0     0.0   
 0.001  0.0893  0.2277  0.2468  0.2643     0.2277  0.0    0.0     0.0   
 0.001  0.0893  0.2277  0.2468  0.2643  …  0.2277  0.0    0.0     0.0   
 0.001  0.0893  0.2277  0.2277  0.2277     0.2684  0.0    0.0     0.0   
 0.001  0.0893  0.2277  0.2277  0.2277     0.2684  0.0    0.0     0.0   
 0.001  0.0893  0.2277  0.2277  0.2277     0.2784  0.0    0.0     0.0   
 0.001  0.0893  0.2277  0.2277  0.2277     0.3025  0.0    0.0     0.0   
 0.001  0.0893  0.2277  0.2277  0.2277  …  0.3128  0.0    0.0     0.0   
 0.0    0.0     0.0     0.0     0.0        0.0     0.001  0.001   0.001 
 0.0    0.0     0.0     0.0     0.0        0.0     0.001  0.1569  0.1569
 0.0    0.0     0.0     0.0

In [23]:
feeder.generators

3-element Array{Generator,1}:
 Generator("g1", 1, 1000.0, 1000.0, 50.0, 1000.0)
 Generator("g2", 12, 0.8, 0.4, 10.0, 5.0)        
 Generator("g3", 7, 0.8, 0.4, 10.0, 5.0)         

In [ ]:
# datadir = ("data/feeder_data/basecase_noneg/")
datadir = ("data/feeder_data/basecase_lv_noneg/")
# datadir = ("data/feeder_data/feeder141/")
# datadir = ("data/feeder_data/feeder141_lv/")

feeder = load_feeder(datadir);

In [ ]:
mu = results_df[:mu_plus] .+ results_df[:mu_minus]
mu = mu[2:end]

b = 1 ./ diag(C)
b = [(x == Inf ? 0 : x) for x in b]

g3 = 2*s*z*e'*R'*mu/sum(b)

g1 = s^2/sum(b) 

delta = results_df[:delta_plus] .+ results_df[:delta_minus]
detlta = delta[2:end]
g2 = sum(delta.*b)/sum(b)

g = -g1 - g2 + g3


In [ ]:
saveres = results_df

In [ ]:
CSV.write("figures/results_uncert.csv", results_df)

In [ ]:
shadow_price(m[:δp][12])

In [ ]:
m.dictList

In [20]:
A = [1 2; 3 4]

2×2 Array{Int64,2}:
 1  2
 3  4

In [31]:
feeder.buses()

MethodError: MethodError: objects of type Array{Bus,1} are not callable
Use square brackets [] for indexing an Array.

In [144]:

Dates.format(Dates.now(), "yymmdd_HHMM")

"190318_1923"